In [ ]:
# %% Install dependencies (Uncomment and run if needed)
# ! pip install accelerate torch
# ! pip install -U git+https://github.com/PrithivirajDamodaran/Gramformer.git
# ! pip install -U pip setuptools wheel
# ! pip install -U spacy
# ! python -m spacy download en_core_web_sm

In [ ]:
# %% Import necessary libraries
import torch
from transformers import pipeline
from gramformer import Gramformer

In [ ]:
# %% Load LLaMA model
model_id = "unsloth/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
# %% Load Gramformer model
gf = Gramformer(models=1, use_gpu=False)  # 1=corrector, 2=detector

In [ ]:
# %% Function to correct grammar using LLaMA
def correct_with_llama(sentence):
    messages = [
        {"role": "system", "content": "Act as a spelling corrector. For every sentence I provide, return only the corrected version without any explanations or additional text."},
        {"role": "user", "content": sentence},
    ]
    outputs = pipe(messages, max_new_tokens=256)
    return (outputs[0]["generated_text"])[-1]['content']

In [ ]:
# %% Test multiple sentences
sentences = [
    "I dont no where my phone is.",
    "She go to the market yesterday.",
    "He do not likes playing football.",
    "The dog barked on the strangers.",
    "Their going too the party later.",
]

In [ ]:
# %% Run corrections
for sentence in sentences:
    print(f"Original: {sentence}")

    llama_correction = correct_with_llama(sentence)
    print("     LLaMA Correction:", llama_correction)

    gramformer_correction = gf.correct(sentence, max_candidates=1)
    for corrected_sentence in gramformer_correction:
        print("Gramformer Correction:",corrected_sentence)


    print("-" * 100)


Original: I dont no where my phone is.
     LLaMA Correction: I don't know where my phone is.
Gramformer Correction: I dont know where my phone is.
--------------------------------------------------
Original: She go to the market yesterday.
     LLaMA Correction: She went to the market yesterday.
Gramformer Correction: She went to the market yesterday.
--------------------------------------------------
Original: He do not likes playing football.
     LLaMA Correction: He does not like playing football.
Gramformer Correction: He does not like playing football.
--------------------------------------------------
Original: The dog barked on the strangers.
     LLaMA Correction: The dog barked at the strangers.
Gramformer Correction: The dog barked at the strangers.
--------------------------------------------------
Original: Their going too the party later.
     LLaMA Correction: They're going to the party later.
Gramformer Correction: Their going to the party later.
----------------------